# Part 1: Access satellite imagery using Python

## Introduction

* We consider optical satellite images from the [Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2), part of the European Union's [Copernicus](https://www.copernicus.eu/en) space programme. 
* We access Sentinel-2 data hosted on the Amazon Web Services (AWS) public cloud and part of the [Earth on AWS](https://aws.amazon.com/earth/) datasets. 
* These datasets can be accessed programmatically via a STAC compliant application programming interface (API).

## The SpatioTemporal Asset Catalog (STAC) specification

<center><img src="https://raw.githubusercontent.com/radiantearth/stac-site/master/images/logo/stac-030-long.png" width=300px></center>

[STAC](https://stacspec.org/en) is an emerging standard for describing geospatial data. Check out the [STAC index](https://stacindex.org) for the available catalogs!

In [ ]:
# If on Colab: uncomment and execute the following line
# ! pip install pystac-client shapely

## Search a STAC catalog

We query the [Earth Search](https://www.element84.com/earth-search/) STAC API endpoint from Python using the library `pystac_client` (library documentation [here](https://pystac-client.readthedocs.io/en/stable/)):

In [ ]:
# STAC API URL 
api_url = 'https://earth-search.aws.element84.com/v1'

Let's query the service for some satellite scenes using the following parameters:

In [ ]:
# collection ID
collection = 'sentinel-2-l2a'

In [ ]:
# AMS coordinates
lon = 4.89
lat = 52.37

In [ ]:
# date range
datetime = '2020-03-20/2020-03-30'

## Access the assets

From the metadata, how do we get to the actual data (the "assets")? 